# Iris Flower - Batch Prediction

# 1) Load the data and model

In [1]:
import pandas as pd
import hopsworks
import joblib

project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/2266


Connected. Call `.close()` to terminate connection gracefully.


## 1.1) Loading the model

In [2]:
mr = project.get_model_registry()
MODEL_VERSION = 1
model = mr.get_model("iris", version=MODEL_VERSION)
model_dir = model.download()
model = joblib.load(model_dir + "/iris_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

## 1.2) Loading the data

In [4]:
feature_view = fs.get_feature_view(name="iris", version=1)

# 2) Batch Inference 

In [6]:
import datetime
from PIL import Image

batch_data = feature_view.get_batch_data()

y_pred = model.predict(batch_data)

y_pred

2022-10-01 23:26:53,181 INFO: USE `nahumsa_featurestore`
2022-10-01 23:26:54,302 INFO: SELECT `fg0`.`sepal_length` `sepal_length`, `fg0`.`sepal_width` `sepal_width`, `fg0`.`petal_length` `petal_length`, `fg0`.`petal_width` `petal_width`
FROM `nahumsa_featurestore`.`iris_1` `fg0`


array(['Versicolor', 'Setosa', 'Virginica', 'Versicolor', 'Virginica',
       'Setosa', 'Virginica', 'Versicolor', 'Versicolor', 'Versicolor',
       'Virginica', 'Virginica', 'Virginica', 'Virginica', 'Virginica',
       'Virginica', 'Versicolor', 'Setosa', 'Versicolor', 'Versicolor',
       'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica',
       'Setosa', 'Setosa', 'Versicolor', 'Setosa', 'Setosa', 'Virginica',
       'Versicolor', 'Virginica', 'Virginica', 'Setosa', 'Setosa',
       'Virginica', 'Setosa', 'Versicolor', 'Setosa', 'Virginica',
       'Versicolor', 'Versicolor', 'Setosa', 'Versicolor', 'Versicolor',
       'Virginica', 'Versicolor', 'Versicolor', 'Versicolor',
       'Versicolor', 'Virginica', 'Versicolor', 'Virginica', 'Versicolor',
       'Setosa', 'Versicolor', 'Virginica', 'Setosa', 'Setosa', 'Setosa',
       'Virginica', 'Setosa', 'Setosa', 'Versicolor', 'Versicolor',
       'Virginica', 'Versicolor', 'Versicolor', 'Virginica', 'Versicolor',
      

Get the last entry and predict the variety of flower

In [10]:
import requests

flower = y_pred[y_pred.size-1]
flower_url = f"https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/{flower}.png"
img = Image.open(requests.get(flower_url, stream=True).raw)  

img.save("../../assets/latest_iris.png")

Getting the expected flower variety

In [11]:
iris_fg = fs.get_feature_group(name="iris", version=1)
df = iris_fg.read()
df

2022-10-01 23:30:19,513 INFO: USE `nahumsa_featurestore`
2022-10-01 23:30:20,543 INFO: SELECT `fg0`.`sepal_length` `sepal_length`, `fg0`.`sepal_width` `sepal_width`, `fg0`.`petal_length` `petal_length`, `fg0`.`petal_width` `petal_width`, `fg0`.`variety` `variety`
FROM `nahumsa_featurestore`.`iris_1` `fg0`


,sepal_length,sepal_width,petal_length,petal_width,variety
0,6.782198,3.440803,3.333318,1.009991,Versicolor
1,4.900000,3.600000,1.400000,0.100000,Setosa
2,5.700000,2.500000,5.000000,2.000000,Virginica
3,5.700000,3.000000,4.200000,1.200000,Versicolor
4,6.700000,3.300000,5.700000,2.100000,Virginica
...,...,...,...,...,...
145,5.800000,2.700000,3.900000,1.200000,Versicolor
146,5.000000,3.400000,1.600000,0.400000,Setosa
147,6.200000,2.200000,4.500000,1.500000,Versicolor
148,5.000000,3.000000,1.600000,0.200000,Setosa


In [12]:
label = df.iloc[-1]["variety"]
label

'Versicolor'

In [13]:
flower_url = f"https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/{flower}.png"
img = Image.open(requests.get(flower_url, stream=True).raw)  

img.save("../../assets/actual_iris.png")

Creating a feature group for writing the predictions of the last entry on the feature store

In [14]:
import pandas as pd

monitor_fg = fs.get_or_create_feature_group(name="iris_predictions",
                                  version=1,
                                  primary_key=["datetime"],
                                  description="Iris flower Prediction/Outcome Monitoring"
                                 )

In [16]:
from datetime import datetime
now = datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")

data = {
    'prediction': [flower],
    'label': [label],
    'model_version': [MODEL_VERSION],
    'datetime': [now],
}

monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/2266/fs/2211/fg/2808


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/2266/jobs/named/iris_predictions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7faf391cc9a0>, None)

In [17]:
history_df = monitor_fg.read()
history_df

2022-10-01 23:35:27,496 INFO: USE `nahumsa_featurestore`
2022-10-01 23:35:28,623 INFO: SELECT `fg0`.`prediction` `prediction`, `fg0`.`label` `label`, `fg0`.`model_version` `model_version`, `fg0`.`datetime` `datetime`
FROM `nahumsa_featurestore`.`iris_predictions_1` `fg0`


,prediction,label,model_version,datetime
0,Versicolor,Versicolor,1,"10/02/2022, 02:33:37"


In [18]:
import dataframe_image as dfi

df_recent = history_df.tail(5)
 
# If you exclude this image, you may have the same iris_latest.png and iris_actual.png files
# If no files have changed, the GH-action 'git commit/push' stage fails, failing your GH action (last step)
# This image, however, is always new, ensuring git commit/push will succeed.
dfi.export(df_recent, '../../assets/df_recent.png')

[1001/233617.405857:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[1001/233617.407704:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[1001/233617.537055:INFO:headless_shell.cc(660)] Written to file /tmp/tmpsaou3dg4/temp.png.


In [19]:
from sklearn.metrics import confusion_matrix

predictions = history_df[['prediction']]
labels = history_df[['label']]

results = confusion_matrix(labels, predictions)
print(results)

[[1]]


In [20]:
from matplotlib import pyplot
import seaborn as sns

# Only create the confusion matrix when our iris_predictions feature group has examples of all 3 iris flowers
if results.shape == (3,3):

    df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                         ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

    cm = sns.heatmap(df_cm, annot=True)

    fig = cm.get_figure()
    fig.savefig("../../assets/confusion_matrix.png") 
    df_cm
else:
    print("Run the batch inference pipeline more times until you get 3 different iris flowers")    

Run the batch inference pipeline more times until you get 3 different iris flowers
